# Challenge
Clean data with Pandas

## Details:
Use the provided JSON dataset and complete the following:
1. Some string values may be, "" instead of null. Set these fields to NaN with numpy before you continue cleaning
2. Clamp publication year to the years of 1800-2026
3. Ensure average_rating, page_count, etc can't have impossible values
4. Normalize dates in "last_checkout"
5. Ensure genres, languages, formats, and publishers have proper capitalization
6. Find and remove duplicate books with the same title/author
7. Drop any rows with empty fields

In [ ]:
import pandas as pd
import numpy as np

